# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from a collection of scientific articles in a MongoDB database.  You will be required to run code to answer the given questions in the Jupyter notebook provided.


Download the dataset articles.json from Blackboard and import it into a collection called 'articles' into the 'coursework' database. The Virtual Machine comes pre-configured with an user "coursework" with password "coursework" with read-write privileges over the coursework database. The admin database is also 'coursework'. 

Each question asks you to implement a function following a certain specification. We have an evaluation script that will check the correctness of your answer. It is essential that:
 
 1. You respect the requested answer format, otherwise, the script will flag your answer as incorrect. If the result is correct, but your formatting is incorrect, you get a penalty of -20% of the mark.
 
 2. Do not insert or delete cells in the notebook you will submit, the evaluation script relies on your answer being in the right cell.
    
    1. If you feel the need of inserting cells to prepare and test your answer, do it in a copy of the notebook. Once you finish an answer that fits in a single cell, transfer it to the notebook that you will submit.
    
    2. You can define auxiliary functions if you want, as long as they are all in the same cell as the main function.
    
    3. If you add print statements for testing, please delete or comment them before submitting.


Answers can make use of the tools you learned so far: MongoDB pipelines and operators, Python code, pandas and networkx. An important aspect of the coursework is to identify in what situations is best to use MongoDB only, and in what situations is best to use multiple tools. A bad choice may make your answer less efficient, and in some cases, not run at all. The marking scheme at the end of the notebook details the expected execution times of your answers. Less efficient answers will get less marks.



 ## Context
    
Bibliometrics Ltd. has hired you, dear data scientist, to analyse the latest articles in Computer Science collected by the DBLP (https://dblp.uni-trier.de/) open bibliographic service and further processed by the AMiner team (https://www.aminer.cn/citation). You have been provided with the articles from 2017 and 2019 and an MS Teams chat window with "The Management", on which you will get the questions you need to answer. 

In [ ]:
#import section
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint
import networkx as nx
import pandas as pd

### 0) 

Before the first question pops in the chat window, you better off set up everything to provide answers.

Task:

Write a function that given the name of a database and the name of a collection returns a pymongo collection object of the collection using the user you just created. Use this function to create a pymongo collection object of the 'articles' collection.


[5 points]

In [ ]:
def get_collection(dbname,collection_name):
    """
    Connects to the server, and returns a collection object
    of the `collection_name` collection in the `dbname` database
    """
    # YOUR CODE HERE
    pass
    client = MongoClient('mongodb://'+dbname+":"+dbname+'@'+'localhost:27017',authSource=dbname)
    db = client[dbname]
    return db[collection_name]

# the collection on which you will work on
articles = get_collection('coursework','articles')



### 1) 

The Management has logged in... they want to know how many articles of each type according to the 'doc_type' property are in the dataset. However, being the competent data scientist you are, you know that you always need to verify your data before start crunching numbers...

### Task:

Write a function that returns the number of articles missing the 'doc_type' property or having it equal to the empty string ("") 

[5 points] 



In [ ]:
def count_missing_doc_types(articles):
    """
    :param articles A PyMongo collection of articles
    :return: int: Number or articles without a a 'doc_type' property or having it equal to the empty string ('') 
    """    
    # YOUR CODE HERE 
    find = {"$or": [{'doc_type': None}, {'doc_type': ''}]}
    doc_count = articles.count_documents(find)
    return doc_count

### 2) 

You inform The Management of how many articles are missing the doc_type property or have it empty. They would like to fix as many as possible before moving forward. They may be able to infer the missing types based on the article publisher, so they ask: what are the publishers of the articles that are missing the doc_type property?

### Task:

Write a function that returns the set of publishers of articles that are missing the doc_type property or have it equal to the empty string. 

[5 points]

In [ ]:
def get_publishers_of_articles_missing_type(articles):
    """
    :param articles PyMongo collection of articles
    :return: Set: set of publishers of articles that are missing the doc_type property, defined as not having the field at all, or having it equal to the empty string.
    """    
    # YOUR CODE HERE
    setOfPublishers = set()
    find = {"$or": [{'doc_type': None}, {'doc_type': ''}]}
    cols = {"_id": 0, "publisher": 1}
    documents = list(articles.find(find, cols))
    for doc in documents:
        if 'publisher' in doc:
            setOfPublishers.add(doc['publisher'])
    return setOfPublishers


### 3)

The Management analysed the set you provided in the previous task and decided that:

 1) Articles missing doc_type  with publisher == 'Springer, Cham' should have 'doc_type' == 'Book'
 
 2) Articles missing doc_type with publisher  == 'RFC Editor' should have 'doc_type' == 'RFC'
 
 3) The remainder of articles missing doc_type should have 'doc_type' == 'N/A'
 
Task:

Write a function that updates the article collection according to The Management's decision.

[5 points]

In [ ]:
def update_doc_types(articles):
    """
    :param articles: PyMongo collection of articles
    :return: PyMongo UpdateResult object 
    """    
    # YOUR CODE HERE
    from pymongo import UpdateMany
    updateRequests = [UpdateMany( {"$and": [{"$or": [{'doc_type': ''}, {'doc_type': None}]} , {'publisher': 'Springer, Cham'}] }, {"$set": {"doc_type": "Book"}}),
                      UpdateMany( {"$and": [{"$or": [{'doc_type': ''}, {'doc_type': None}]} , {'publisher': 'RFC Editor'}]},{"$set": {"doc_type": "RFC"}}),
                      UpdateMany( {"$or": [{'doc_type': ''}, {'doc_type': None}]} , {"$set": {"doc_type": "N/A"}})]
    
    updateResult = articles.bulk_write(updateRequests, ordered=True)

    return updateResult.bulk_api_result


### 4)

You are finally ready to answer the original question: What is the distribution of document types in the dataset?

Task:

Write a function that returns a dictionary with doc_types as keys and the number of articles of each type as values.

e.g:

{'Conference' : 4566 , 'N/A' : 7992 ...}



[5 points]


In [ ]:
def get_types_distribution(articles):
    """
    :param articles: PyMongo collection of articles
    :return: Dictionary with article types as keys, and number of articles of type as value
    """ 
    # YOUR CODE HERE
    group = {'$group': {'_id': '$doc_type', 'noOfDocs': {'$sum': 1}}}
    documents = list(articles.aggregate([group]))
    keyList = []
    valueLst = []
    for doc in documents:
        keyList.append(doc["_id"])
        valueLst.append(doc["noOfDocs"])
    docTypeCountDict = dict(zip(keyList, valueLst))
    return docTypeCountDict


###  5)

Do longer articles (in number of pages) have more references?

Task:

Return an histogram dictionary with the following specification:

{
"1-5" : Average references of articles between 1 and 5 pages inclusive

"6-10" : Average references of articles between 6 and 10 pages inclusive

"11-15" : Average references of articles between 11 and 15 pages inclusive

"16-20" : Average references of articles between 16 and 20 pages inclusive

"21-25" : Average references of articles between 21 and 25 pages inclusive

"26-30" : Average references of articles between 26 and 30 pages inclusive

">30" : Average references of articles with more than 30 pages
}

A fellow data scientist of your team has found that some articles have no 'references' field, while others have unusually large page numbers. The Management decides that for this task you should

* Ignore articles that are missing the 'references' field
* Ignore articles with page numbers greater than 6 digits
* Ignore articles with page numbers that cannot be parsed to integers (e.g. 900467-12)
* Remember articles that start and end at the same page have 1 page, not zero!


[10 points]


In [ ]:

def length_vs_references(articles):
    """
    :param collection A PyMongo collection object
    :return dictionary in the form described above
    """    
    histDict= {}
    checkReferences = {'$match': {'references': { '$exists' : True}, 'pageStatus' : True }}
    projectPageDetails = {'$project' : { 'pageStatus' : { '$and' : [{'$regexMatch': {'input': "$page_start",'regex': '^[0-9]{1,6}$','options': "m"}},
    {'$regexMatch': {'input': "$page_end",'regex': '^[0-9]{1,6}$','options': "m"}}]}, 'references' : '$references','page_start' : '$page_start', 'page_end' : '$page_end' }}
    projectReferencDetails= {'$project' : {'pages': {'$add': [{'$subtract': [{'$toInt': '$page_end'}, {'$toInt': '$page_start'}]}, 1]}, 'refcount' : { '$size' : '$references'} }}
    bucketByRange = {'$bucket': {'groupBy': "$pages",'boundaries': [ 1, 6, 11, 16, 21, 26, 31 ],'default': ">30",'output': {    'average':  { '$avg':'$refcount'} } } }
    projectValues = {'$project' : {'_id':1, 'average' : { '$round' : ['$average',3]}}}
    documents = articles.aggregate([projectPageDetails,checkReferences,projectReferencDetails,bucketByRange,projectValues])

    for doc in documents:
        try:
            histDict[str(doc['_id']) +'-'+ str(doc['_id']+4)]= doc['average']
        except:
            histDict[str(doc['_id'])] = doc['average']
    return histDict



###  6)

Being the competent data scientist you are, you remember that before sending the results to The Management you  should be verify that they are meaningful and not affected by data quality. Can you trust these averages are not affected by outliers?

Task:

Write a function to return for each article length range a list of outliers, that is,  articles with a z-score of number of references greater or equal than 3 ((https://www.statisticshowto.com/probability-and-statistics/z-score/)) 

Note that we say "for each article length range", therefore, z-score needs to be computed using the mean and stdev  of the provided article lengths. For Standard Deviation in Mongo, use https://docs.mongodb.com/v4.4/reference/operator/aggregation/stdDevPop/

Example outpust is provided in a comment in the function

[10 points]


In [ ]:
def get_reference_outliers(articles):
    """
    :param articles A PyMongo collection object
    :return Dictionary of the form described below:
     {"1-5" : {'outliers':[  
             {          
            id: article_id,           
            num_references: number of references,           
            z-score: z-score
          }, ... and so on...], 
     "6-10" : {'outliers':[  
             {          
            id: article_id,           
            num_references: number of references,           
            z-score: z-score
          }, ... and so on...] ,
    .... and so on.....
       }  
    """ 
    checkReferences = {'$match': {'references': { '$exists' : True}, 'pageStatus' : True }}
    projectPageDetails = {'$project' : { 'pageStatus' : { '$and' : [{'$regexMatch': {'input': "$page_start",'regex': '^[0-9]{1,6}$','options': "m"}},
    {'$regexMatch': {'input': "$page_end",'regex': '^[0-9]{1,6}$','options': "m"}}]}, 'references' : '$references','page_start' : '$page_start', 'page_end' : '$page_end', 'id' : '$id' }}
    projectReferencDetails= {'$project' : {'pages': {'$add': [{'$subtract': [{'$toLong': '$page_end'}, {'$toLong': '$page_start'}]}, 1]}, 'refcount' : { '$size' : '$references'}, 'id' : '$id' }}
    bucketByRange = {'$bucket': {'groupBy': "$pages",'boundaries': [ 1, 6, 11, 16, 21, 26, 31 ],'default': ">30",'output': { 'stddev': { '$stdDevPop':'$refcount'}, 'mean' : {'$avg': '$refcount'} ,'count' :{'$sum': 1} ,  'articles' : { '$push': { 'id' : '$id' , 'refcount' : '$refcount'}}} } }
    unwindArticles= {'$unwind': {'path': '$articles'}}
    projectZscore ={ '$project' : { '_id' : 1 , 'mean' : '$mean' , 'stdDev' : '$stddev','zscore' : { '$divide': [ {'$subtract': [{'$toInt': '$articles.refcount'}, {'$toInt': '$mean'}]}, {'$toInt': '$stddev'}]}, 'refcount' : '$articles.refcount', 'id' :'$articles.id'}}
    groupByRange ={'$group' : {'_id' : '$_id' , 'outliers' :{ '$push' : { 'id' : '$id' , 'num_references' : '$refcount', 'z-score' : { '$round' : ['$zscore',3]} }}}}
    matchZscore = {'$match': {'zscore' : {'$gte' : 3}}}
    sortByRange = { '$sort' : {'_id' : 1}}
    documents = articles.aggregate([projectPageDetails,checkReferences,projectReferencDetails,bucketByRange,unwindArticles,projectZscore,matchZscore,groupByRange,sortByRange], allowDiskUse = True)

    zScoreDict = {}
    for doc in documents:
      outliersDict = {}
      try:
        outliersDict['outliers'] = doc['outliers']
        zScoreDict[str(doc['_id']) +'-'+str(doc['_id']+4)] = outliersDict
      except:
        outliersDict['outliers'] = doc['outliers']  
        zScoreDict[str(doc['_id'])] = outliersDict
   
    return zScoreDict
    



### 7) 

What are the collaborators of an author? 

Task:

Write a function that receives as input an author id and returns a list of authors that have at least one article co-authored with the input author. 
    
Notes:
* Assume authors are uniquely identified by their ids.
* Affiliations are in the 'org' field of the author subdocuments, you need to rename that field in your output to conform to the expected format.
* Sometimes affiliations come together in the same string value instead of different documents e.g. "University of Southampton and University of Bournemouth", consider this case as a single affiliation.
* Remember a dictionary does not have an ordering in its keys
* Some articles list authors only with name, without affiliation. Include this in your answer as a dictionary with a single key 'name'
* Example output is provided in the definition of the function


[8 points]




In [ ]:


def get_collaborators(articles,author_id):
    """
    Input: articles: PyMongo collection of articles
        author_id : id of author
    Output: List of collaborators. 
    Example output:
    [
    {'id': 2942936634,  'names': [
           {'name': 'Lior Kovalio',
           'affiliation': 'The Hebrew Univ. of Jerusalem, Jerusalem, Israel'},
           {'name' : 'Lior Kovalio'}
           ]
    },
    {'id': 2231056490, 'names': [
           {'name': 'Gustavo F. Tondello', 
           'affiliation': 'University of Waterloo'},
           { 'name': 'Gustavo F. Tondello', 
           'affiliation': 'Federal University of Santa Catarina'}
           ]
    },
    {'id': 19031441 , 'names': [
                  {'affiliation': 'Hebrew University of Jerusalem, Israel / '
                            'Microsoft Research, Israel#TAB#',    
                    'name': 'Noam Nisan'},
                   {'affiliation': 'The Hebrew University of Jerusalem, Rachel & '
                            'Selim Benin School of Computer Science & '
                            'Engineering and Federmann Center for the Study of '
                            'Rationality, Israel',
                     'name': 'Noam Nisan'} 
                            ]
     }
     ]    
    """
    #Your code here
    findAuthor = {'$match': {'authors.id': author_id}}
    projecrAtricleAuthorDtls = {'$project' : {'articleId':'$id', 'authors' : '$authors','coauthors' : '$authors.id'}}
    unwindAuthors = {'$unwind' : {'path' : '$authors'}}
    unwindCoauthors = {'$unwind' : {'path' : '$coauthors'}}
    removeSelfAuthor = {'$match' : {'authors.id' : { '$ne' : author_id} }}
    groupByAuthors = {'$group' : { '_id' : '$authors.id', 'names' : { '$addToSet' : { 'affiliation' : '$authors.org' ,'name' : '$authors.name'}}}}
    documents = list(articles.aggregate([findAuthor,projecrAtricleAuthorDtls,unwindAuthors,unwindCoauthors,removeSelfAuthor,groupByAuthors]))   
    
    return documents

    


### 8)

The Management anticipates that several queries based on authors and their connections will be required in the near future. You have been asked to take the necessary steps to ensure these queries are easier to express and faster to perform.

Task : Create a collection named authors containing for each author a document with the following fields:
 * _id: author's id
 * name_affiliations: array of pairs (name ,affiliation) associated to _id
 * articles: array of artilcle ids authored by author with id = _id
 * collaborators: array of ids of collaborators, other authors that have written at least one article with _id

We provide an authors.json file with a sample of the expected output. This file is in a slightly different format than articles.json, to load it, you need to include the --jsonArray option to your mongoimport command (see the documentation: https://docs.mongodb.com/database-tools/mongoimport/#cmdoption-mongoimport-jsonarray)

[10 points]

In [ ]:
def create_authors_collection(articles):
    """
    :param: articles collection
    : output: Pymongo cursor of the newly created authors collection
    {
     '_id': author's id ,
     'name_affiliations': array of  {name: name, affiliation: affiliation} associated to author's id ,
      'articles': array of article_ids authored by this author id ,
      'collaborators' : Array of ids of collaborators
    }
     Sample in authors.json
    """
    projectRequiredDetails = {'$project' : {'_id' : 0, 'articles' : '$id', 'coauthors' : '$authors.id' , 'author' : '$authors'}}
    unwindAuthors ={'$unwind': {'path' : '$author'}}
    unwindCoauthors = {'$unwind' : {'path': '$coauthors'} }
    projectCorrectNames = {'$project' : { 'articles':1,'coauthors' : 1,'authorid': '$author.id', 'name' : '$author.name','affiliation' : '$author.org'}}
    removeSelfAuthors = { '$project' : {'articles' : 1, 'authorid' : 1, 'name': 1,'affiliation' : 1,'coauthors':{ '$cond':{'if':{'$eq':['$coauthors','$authorid']},'then':'$REMOVE','else':'$coauthors'} } }}
    groupByAuthorId = {'$group' : {'_id' : '$authorid', 'articles' : {'$addToSet' : '$articles'}, 'name_affiliations' : {'$addToSet':{'name':'$name','affiliation':'$affiliation'}},'collaborators':{'$addToSet':'$coauthors'}}}
    createCollection = {'$out' : 'authors'}
    documents = articles.aggregate([projectRequiredDetails,unwindCoauthors,unwindAuthors,projectCorrectNames,removeSelfAuthors,groupByAuthorId,createCollection],allowDiskUse = True)

    return documents
    
create_authors_collection(articles)
authors = get_collection('coursework','authors')

### 9)


The Management is investigating the ties of authors with collaborators of different countries. They would like to know for a given author, to how many authors of the same country they could reach through colleagues of that country

Task:

Given an author id and the name of a country, return the ids of the authors that match the following conditions:
 1) have at least one affiliation that includes the input country name
 2) there is a path between this author and the input author where all nodes match condition (1)
 3) at a distance of at most 3 from the input author id in the graph of co-authorship


Notes:

* Do not include authors without affiliation, or affiliations that do not explicitly include the name of the country. Consider those as not matching the query. 

* If you are stuck with question 8) you can use the authors.json sample provided for question 8 for development of questions 9). We will mark questions 9) by running them on the actual answer of Q8. Do note the actual answer of Q8 is much larger than the sample, take this in consideration when designing your solutions for Q9.

* Watch out for authors with many articles (thus, a large network of collaborators)... They may consume all your memory...


[10 points]



![Q8-example.png](attachment:Q8-example.png)


In [ ]:

def get_network(authors,author_id,country):
    """
    Input: Authors collection, author_id, String country
    Output: List of ids of authors at three or less hops of distance with an affiliation that includes 'country'
    """
    #Your code here
    matchAuthorId = {'$match' :  {'_id' :  author_id}}
    graphLookup = {'$graphLookup' : {'from': "authors",  'startWith': "$collaborators", 'connectFromField' : 'collaborators' , 'connectToField' : '_id', 'as' : 'coauthorsDetails' ,   'maxDepth': 2 , 'depthField' : "depth", 'restrictSearchWithMatch' : {'name_affiliations.affiliation'  : {'$regex' :  country, '$options' : 'i'}}}}
    unwindCoAUthors = {'$unwind' :  {'path' : '$coauthorsDetails'}}
    projectAuthorIds = { '$project' : {'_id': 0, 'authorid' : '$coauthorsDetails._id' }}
    documents = authors.aggregate([matchAuthorId,graphLookup,unwindCoAUthors,projectAuthorIds])
    authorIdLst= []
    for doc in documents:
        authorIdLst.append(doc['authorid'])
    return authorIdLst



### 10)

The Management wants to know what articles are more important in the field of Machine Learning (look at the "fos" field for that). They know that papers in the references field may be from years previous to the ones in your dataset, so they would also like to know what is the most important paper within that subset published in each of the years you have in your dataset (2017,2018,2019). They want to use in-degree centrality as measure of importance.

Task:

Write a function that filters the subset of "Machine learning" articles and from it, compute the article with the highest in-degree centrality, both overall, and for articles published in each of the 2017, 2018 and 2019 years. 

Example output is provided as a comment in the function.


[10 points]

In [ ]:

def machine_learning_central(articles):
    """
    Input: Articles collections.
    Output : {
       'overall' : (id_article_highest_indegree_centrality),
       '2017' : id_article_highest_indegree_centrality published year 2017,
       '2018' : id_article_highest_indegree_centrality published year 2018,
       '2019' : id_article_highest_indegree_centrality published year 2019,
    }
    In the case that for a given year, no article has been referenced (and thus, all centrality values are zero), 
    put None as the value of that year, example:
    {
       'overall' : id_article_highest_indegree_centrality,
       '2017' : id_article_highest_indegree_centrality of year 2017,
       '2018' : id_article_highest_indegree_centrality of year 2018,
       '2019' : None,
    }
    """
    #Your code here
    machineLearningDict = {}
    checkFos = {'$match': {"$and": [{'fos': { '$exists' : True} }, {'references': { '$ne': None}}]}}
    unwindFos = {'$unwind': {'path' : '$fos'}}
    filterMachinieLearningArticles = {'$match' : {'fos.name' : {'$regex' : 'machine learning' , '$options' : 'i'}}}
    groupByYear = {'$group' : { '_id' : '$year','articleDetails' : {'$addToSet' : { 'articleId' : '$id', 'references' : '$references'}}}}
    unwindArticleDetails = {'$unwind' :{'path' : '$articleDetails'}}
    unwindReferencesDetails = {'$unwind' :{'path' : '$articleDetails.references'}}
    projectDetails = {'$project' : {'_id':0, 'year':'$_id', 'articleId' : '$articleDetails.articleId', 'references' : '$articleDetails.references'}}
    documents = list(articles.aggregate([checkFos,unwindFos,filterMachinieLearningArticles,groupByYear,unwindArticleDetails,unwindReferencesDetails,projectDetails]))
    articlesDataFrame = pd.DataFrame(documents)
    machineLearningDict['overall'] = calculateInDegreeCentrality(articlesDataFrame,'references','articleId')
    machineLearningDict['2017'] = calculateInDegreeCentrality(articlesDataFrame[articlesDataFrame['year'] == 2017],'references','articleId')
    machineLearningDict['2018'] = calculateInDegreeCentrality(articlesDataFrame[articlesDataFrame['year'] == 2018],'references','articleId')
    machineLearningDict['2019'] = calculateInDegreeCentrality(articlesDataFrame[articlesDataFrame['year'] == 2019],'references','articleId')
    return machineLearningDict

         
def calculateInDegreeCentrality(articlesDataFrame,references,articleId):
   diGraph = nx.from_pandas_edgelist(articlesDataFrame,source=articleId,target=references, edge_attr=True, create_using=nx.DiGraph())
   centralityDegree = nx.in_degree_centrality(diGraph)
   max_value = max(centralityDegree.values())
   max_key = [i for i in centralityDegree.keys() if centralityDegree[i]==max_value]
   return max_key[0]

## Task 2
This task will assess your ability to use the Hadoop Streaming API and MapReduce to process data. For each of the questions below, you are expected to write two python scripts, one for the Map phase and one for the Reduce phase. You are also expected to provide the correct parameters to the `hadoop` command to run the MapReduce process. Write down your answers in the specified cells below.

You will use the same dataset of articles that you used for task 1.

To help you, `%%writefile` has been added to the top of the cells, automatically writing them to "mapper.py" and "reducer.py" respectively when the cells are run.

No need to return a Python dictionary, the expected output here is the output file of Hadoop, named "output" for question1 and "output2" for question (do not change that part of the question)

### 1) 

Answer Question 2) of task 1 using the MapReduce paradigm (repeated below for your convenience)

Return the set of publishers of articles that are missing the doc_type property or have it equal to the empty string. 

[8 points]

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
# Answer for mapper.py
import json
import sys

for eachJsonDocument in sys.stdin:
    publisher=''
    eachArticle = json.loads(eachJsonDocument)
    if (('doc_type' not in eachArticle or eachArticle['doc_type'] == '') and ( 'publisher' in eachArticle)):
        try:
            publisher = eachArticle['publisher']
        except:
            continue
        print(publisher+'\t1')

In [ ]:
%%writefile reducer.py
#!/usr/bin/env python
# Answer for reducer.py
import json
import sys
setOfPublishers = set()
inputPairs = sys.stdin.readlines()

for row in inputPairs:
    keyValuePair = row.split('\t',1)

    if ( keyValuePair[0] not in setOfPublishers):
        setOfPublishers.add(keyValuePair[0])
        print (keyValuePair[0])

In [ ]:
%%bash
# Hadoop command to run the map reduce.
rm -r output
hadoop-standalone-mode.sh
hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-input ~/datasets/articles.json \
-mapper ./mapper.py \
-reducer ./reducer.py \
-output output

In [ ]:
#Expected output format, list of publishers, one per line
#Elsevier India
#Springer Cham
#UK Academy of Sciences
#....



### 2) 
Answer Question 5) of task 1 using the MapReduce paradigm (repeated below for your convenience)

Do longer articles (in number of pages) have more references?

Task:

Return an histogram dictionary with the following specification:

{
"1-5" : Average references of articles between 1 and 5 pages inclusive

"6-10" : Average references of articles between 6 and 10 pages inclusive

"11-15" : Average references of articles between 11 and 15 pages inclusive

"16-20" : Average references of articles between 16 and 20 pages inclusive

"21-25" : Average references of articles between 21 and 25 pages inclusive

"26-30" : Average references of articles between 26 and 30 pages inclusive

">30" : Average references of articles with more than 30 pages
}

A fellow data scientist of your team has found that some articles have no 'references' field, while others have unusually large page numbers. The Management decides that for this task you should

* Ignore articles that are missing the 'references' field
* Ignore articles with page numbers greater than 6 digits
* Ignore articles with page numbers that cannot be parsed to integers (e.g. 900467-12)

Clarifications:
*) Articles that start and end at the same page have 1 page, not zero!


[12 points]

In [ ]:
%%writefile mapper2.py
#!/usr/bin/env python
# Answer for mapper.py
import json
import sys

for eachJsonDocument in sys.stdin:
    eachArticle = json.loads(eachJsonDocument)
    if ( ('references' in eachArticle) and 
        ( ('page_start' in eachArticle) and (eachArticle['page_start'].isnumeric()) and( len(eachArticle['page_start']) <=6 ) ) and 
         ( ('page_end' in eachArticle) and (eachArticle['page_end'].isnumeric()) and( len(eachArticle['page_end']) <=6 ) )
        ) :
        noOfPage = (int(eachArticle['page_end']) - int (eachArticle['page_start'])) + 1
        if( noOfPage >= 1 and noOfPage <=5):
            print ( '1-5'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >= 6 and noOfPage <=10):
            print ( '6-10'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >= 1 and noOfPage <=15):
            print ( '11-15'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >= 16 and noOfPage <=20):
            print ( '16-20'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >= 21 and noOfPage <=25):
            print ( '21-25'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >= 26 and noOfPage <=30):
            print ( '26-30'+'\t'+str(len(eachArticle['references'])))
        elif (noOfPage >30):
            print ( '>30'+'\t'+str(len(eachArticle['references'])))

In [ ]:
%%writefile reducer2.py
#!/usr/bin/env python
# Answer for reducer.py
import json
import sys

from collections import defaultdict
accumulatorRefLength = defaultdict(lambda: 0)
accumulatorCount = defaultdict(lambda: 0)
rangeDict = {}

inputPairs = sys.stdin.readlines()

for row in inputPairs:
    keyValuePair = row.split('\t',1)
    accumulatorRefLength[keyValuePair[0]] =  accumulatorRefLength[keyValuePair[0]] + int(keyValuePair[1].strip())
    accumulatorCount[keyValuePair[0]] = accumulatorCount[keyValuePair[0]]+1

for key,value in accumulatorRefLength.items():
    accumulatorRefLength[key] = round(value/accumulatorCount[key],3)
accumulatorRefLength = {k: v for k, v in sorted(accumulatorRefLength.items(), key=lambda item: item[1])}

for key,value in accumulatorRefLength.items():
    print(key+'\t'+str(value))


In [ ]:
%%bash
# Hadoop command to run the map reduce.
rm -r output2
hadoop-standalone-mode.sh
hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper2.py,reducer2.py \
-input ~/datasets/articles.json \
-mapper ./mapper2.py \
-reducer ./reducer2.py \
-output output2


In [ ]:
#Expected key-value output format:
#1-5    average
#6-10   average
#... and so on

### Marking scheme

- The result provided is correct: An incorrect answer will have between 0% and 40% of the mark depending on the nature of the mistake. Questions where there was only one answer possible will have 0%, questions where the result is correct in some cases and not others will be marked at 20% or 40%. Feel free to create as many notebooks as you want for experimenting and transcribe your final answer to the one you submit.

- The result is provided in the expected format and output: 20% will be deducted to correct results that are not in the expected format

- Efficiency of the answer: Measured in terms of execution time. There are many ways to reach the correct result, some of them are more efficient than others, some are more straight forward than others. 

- Tables below detail the percentage of mark you get according to the efficiency of the answer, each cell shows the maximum time allowed to get the mark in the corresponding row. Answers that take more time than the time in the 60% column will be declared timeout and get zero points. (run the cells for showing the tables)





| Task1 | Points |  100% | 80% | 60% | 
| --- | --- | --- | --- | --- | 
| q1 | 5 | 5sec |  10 sec | 2minutes | 
|  q2 | 5 | 5sec   | 10 sec   | 2minutes   |
|  q3 | 5 |  10sec  | 40sec  |  5minutes  |
|  q4 | 5 | 5sec   | 10sec   | 2minutes   |
|  q5 | 10 | 10sec   | 60sec  | 8minutes   |
|  q6 | 10  |  15sec   | 90sec   | 10minutes   |
|  q7 | 10 | 15sec    | 90sec   | 10minutes   |
|  q8 | 10 | 90 sec    | 5minutes  | 10minutes  | 
|  q9 (worst case) | 10   | 20sec   |  100 sec  | 10 minutes  |
|  q10| 10  |  60sec  | 3minutes   | 10minutes   |

| Task2 | points | 100% | 80% | 60% | 
| --- |--- | --- | --- | --- | 
| q1 | 8 | 90sec |  5minutes | 10minutes | 
|  q2| 12  | 90sec   | 5minutes   | 10minutes   |